# About overriding the `input` function in Jupyter Notebook

Let's experiment with the substitution of the built-in function `input`
(for possible reasons for this see [the comments on this tweet](https://twitter.com/vitalizzare/status/1342463345690087424), as well as [the reflections at the end of the article](#use_case))

In [1]:
import builtins
builtins.input = lambda prompt, _input = input: print('Hello world!') or _input(prompt)
input('Say hello: ')

Hello world!
Say hello: Hello Jupyter!!!


'Hello Jupyter!!!'

Let's see how the built-in `input` behaves now

In [2]:
input('Say it again: ')

Say it again: Why there's no 'hello world' now???


"Why there's no 'hello world' now???"

Function replacement worked within one cell. Why?
Let's check if the object pointed to by the `input` identifier changes when moving from one cell to another.

In [3]:
first_cell = id(input)

In [4]:
second_cell = id(input)

In [5]:
first_cell, second_cell, first_cell == second_cell

(139711962051520, 139711936650432, False)

Well, it is really changing. In two different cells, we see two different `input` objects. Let's check if this is common to other built-in functions, for example `len`.

In [6]:
first_cell = id(len)

In [7]:
second_cell = id(len)

In [8]:
first_cell, second_cell, first_cell == second_cell

(139712020859216, 139712020859216, True)

`len` when moving from one cell to another is the same object.
This is also true for `print`, ` hash`, `id`, ` locals`, `globals`, ` eval`, etc.
What's wrong with `input`?

In [9]:
input.__name__

'raw_input'

In [10]:
input.__module__

'ipykernel.kernelbase'

As we see this function is not from the `builtins` module. Let's try to get the code.

In [11]:
import inspect
print(inspect.getsource(input))

    def raw_input(self, prompt=''):
        """Forward raw_input to frontends

        Raises
        ------
        StdinNotImplentedError if active frontend doesn't support stdin.
        """
        if not self._allow_stdin:
            raise StdinNotImplementedError(
                "raw_input was called, but this frontend does not support input requests."
            )
        return self._input_request(str(prompt),
            self._parent_ident,
            self._parent_header,
            password=False,
        )



As you can see, this function is a method of some class. Moreover, it is a wrapper around the `self._input_request` method, which, apparently, is the real `input`. Let's see in which class our method is declared and in which module this class is located.

In [12]:
input.__self__.__class__.__name__

'IPythonKernel'

In [13]:
input.__self__.__class__.__module__

'ipykernel.ipkernel'

Let's import this class and try to replace its `_input_request` method

In [14]:
from ipykernel.ipkernel import IPythonKernel

ipython_input = IPythonKernel._input_request

def my_input(*args, **kwargs):
    print('Hello world!')
    return ipython_input(*args, **kwargs)

IPythonKernel._input_request = my_input

Let's check what do we have now.

In [15]:
input()

Hello world!
Well, hello there!


'Well, hello there!'

In [16]:
input('Say hello again: ')

Hello world!
Say hello again: Good, we have 'hello world' again!


"Good, we have 'hello world' again!"

In [17]:
input("Let's do it one more time: ")

Hello world!
Let's do it one more time: And again!


'And again!'

# Looking for authentic `input` from builtins module

Well, we have found what we really need to replace. But where is the `input` method from the `builtins` module? 
When opening a Jupyter Notebook, we are automatically provided with the `get_ipython` function. What does this function do?

In [18]:
help(get_ipython)

Help on method get_ipython in module IPython.core.interactiveshell:

get_ipython() method of ipykernel.zmqshell.ZMQInteractiveShell instance
    Return the currently running IPython instance.



This function provides access to a running kernel. Let's see if there is something inside of the kernal namespace containing in its name the word 'input'.

In [19]:
instance = get_ipython()

In [20]:
for item in dir(instance):
    if 'input' in item:
        print(item)

_last_input_line
auto_rewrite_input
extract_input_lines
input_splitter
input_transformer_manager
input_transformers_cleanup
input_transformers_post
raw_input_original
set_next_input
show_rewritten_input


Let's check what is `raw_input_original`

In [21]:
type(instance.raw_input_original)

builtin_function_or_method

In [22]:
instance.raw_input_original.__name__

'input'

In [23]:
instance.raw_input_original.__module__

'builtins'

In [24]:
help(instance.raw_input_original)

Help on built-in function input in module builtins:

input(prompt=None, /)
    Read a string from standard input.  The trailing newline is stripped.
    
    The prompt string, if given, is printed to standard output without a
    trailing newline before reading input.
    
    If the user hits EOF (*nix: Ctrl-D, Windows: Ctrl-Z+Return), raise EOFError.
    On *nix systems, readline is used if available.



Yes, this is original `input` from the `builtins` module. And it doesn't seem to be used. But back to the `input` function that Jupyter provides.

<a id="use_case"></a>
## How can we use the acquired knowledge

A use case I've been thinking about for a some time is a context manager for passing the same response to `input`.

In [25]:
from ipykernel.ipkernel import IPythonKernel
from contextlib import contextmanager

@contextmanager
def repeat(answer: str):
    try:
        ipython_input = IPythonKernel._input_request
        
        def mock_input(*args, **kwargs):
            return answer
        
        IPythonKernel._input_request = mock_input
        yield
    
    finally:
        IPythonKernel._input_request = ipython_input

In [26]:
with repeat('yes'):
    answer_1 = input('Continue?')
    answer_2 = input('Continue?')
    answer_3 = input('Continue?')
    
print(answer_1)
print(answer_2)
print(answer_3)

yes
yes
yes


Let's check if everything was restored correctly (remember that earlier we have already changed `input` so that it greets us with <mark>Hello world!</mark>)

In [27]:
input('Try to say something: ')

Hello world!
Try to say something: Well, everything seems to be allright.


'Well, everything seems to be allright.'

It works. What it looks like if we apply a context manager to a process with input.

In [28]:
def process():
    print('Starting the process...')
    print('Working...')
    for _ in range(3):
        print('Paused.')
        answer = input('Continue? (Y)es|(N)o :')
        if answer == 'Y':
            print('Resume and continue the process.')
        else:
            print('Breaking the process.')
            break
    print('Done.')

This is how the process works *without* a context manager.

In [29]:
process()

Starting the process...
Working...
Paused.
Hello world!
Continue? (Y)es|(N)o :Y
Resume and continue the process.
Paused.
Hello world!
Continue? (Y)es|(N)o :Y
Resume and continue the process.
Paused.
Hello world!
Continue? (Y)es|(N)o :Y
Resume and continue the process.
Done.


So, we manually entered 'Y' three times. Now let's start the process with the substitution of the input function.

In [30]:
with repeat('Y'):
    process()

Starting the process...
Working...
Paused.
Resume and continue the process.
Paused.
Resume and continue the process.
Paused.
Resume and continue the process.
Done.


Let's check how `input` works after exiting the context manager

In [31]:
input('Check the state of input: ')

Hello world!
Check the state of input: Okey


'Okey'

Finally, let's clean up after ourselves!

In [32]:
IPythonKernel._input_request = ipython_input

In [33]:
input('The End. ')

The End. Bye!


'Bye!'